## Mount Google Drive storage and import the data

In [ ]:
# Mount Google Drive storage
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import training data
# First row = labels as strings, every other row = (answer,pixel data array)
data_train = pd.read_csv( \
                "/content/drive/MyDrive/google_colaboratory/data/train.csv", \
                header=0,dtype=float).values

# Import test data
data_test = pd.read_csv( \
                "/content/drive/MyDrive/google_colaboratory/data/test.csv", \
                header=0,dtype=float).values


## Single-layer (softmax) and multi-layer (N*ReLU+softmax) perceptrons


### Display an image from the training data

In [ ]:
# Pick an image
image_num = 16
label = data_train[image_num][0]
image = data_train[image_num][1:].reshape(28,28)

# Display the image and its label
print("This number is: ",int(label))
plt.imshow(image, cmap="gray")
plt.axis("off")
plt.show()


### Define input/output and functions

In [ ]:
# DEFINE MODEL'S PARAMETERS, INPUTS AND OUTPUTS

# Inputs:
# X = array(num_samples,num_features)
# (for each sample, pixel values in range [0,1], flattened 28x28-pixel image)
X = np.array(data_train[:,1:]/255.0)
# Mean subtraction: subtract average across dataset, to make features balanced
X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0)+10**(-9))

# Outputs:
# Y = array(num_samples)
# (for each sample, correct class = digit)
Y = np.array(data_train[:,0].astype(int))

# Number of classes for classification (digits -> 10 possible classes)
num_classes = 10

# Number of samples (data points)
num_samples = X.shape[0]

# Number of features (pixels in a single sample)
num_features = X.shape[1]

# Number of hidden layers
num_layers = 3

# Dimension of a hidden layer
num_hidden = 256

# Generate array of initial parameters
def generate_parameter(dim_1, dim_2):
    W = np.sqrt(2.0/dim_1)*np.random.randn(dim_1,dim_2)
    b = np.full((1, dim_2), 0.01)
    return [W, b]
if num_layers==1: parameters_init = [generate_parameter(num_features,num_classes)]
else:
    parameters_init = [generate_parameter(num_features,num_hidden)]
    for layer in range(2,num_layers):
        parameters_init.append(generate_parameter(num_hidden,num_hidden))
    parameters_init.append(generate_parameter(num_hidden,num_classes))

#-------------------------------------------------------------------------------

# DEFINE FUNCTIONS

# ReLU activation function (used for hidden layers): removes negative scores
def relu(Z): return np.maximum(0,Z)
def relu_derivative(Z): return (Z>0).astype(float)

# SOFTMAX activation function (used for output layer):
# transforms arbitrary-valued scores (logits) into probabilities
# Z = array(num_samples,num_classes)
# (for each sample, arbitrary-valued scores for each class)
# softmax(Z) = array(num_samples,num_classes)
# (for each sample, probability of belonging to a certain class)
def softmax(Z):
    exp_z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Function that converts outputs (labels) into corresponding probability vectors
# Y = array(num_samples)
# one_hot(Y) = array(num_samples,num_classes)
# (for each sample, probability of belonging to a certain class)
def one_hot(Y): return np.eye(num_classes)[Y]

# LOSS FUNCTION - cross-entropy: calculates the punishment for a bad prediction
# = LOG(cummulative probability of predicting all outputs)
def cross_entropy(probs_pred, probs_true):
    # 10**(-9) is needed to avoid LOG(0) error
    return -np.sum(probs_true * np.log(probs_pred + 10**(-9))) / num_samples
def cross_entropy_derivative(probs_pred, probs_true):
    return (probs_pred - probs_true) / num_samples

# FORWARD PROPAGATION

# Single-layer forward propagation
def forward_SL(X, W, b, activation_function):
    # Calculate arbitrary-valued scores
    Z = X.dot(W) + b
    # Apply the activation function
    Y = activation_function(Z)
    return [Z, Y]

# Multi-layer forward propagation
# Examples:
# params_array = [[W1,b1],[W2,b2],...]
def forward_ML(X_init, parameters):
    forward_parameters, X = [[0, X_init]], X_init
    # For every layer: calculate forward propagation and update initial vector
    for index, parameter in enumerate(parameters):
        # softmax is only applied at the outer level
        activation_function = relu if index!=(num_layers-1) else softmax
        forward_parameter = forward_SL(X,*parameter,activation_function)
        X = forward_parameter[1]
        forward_parameters.append(forward_parameter)
    # Export list of [Z, Y] for all layers:
    # index 0 corresponds to the initial data, so len(forward_params)=num_layers+1
    return forward_parameters

# BACKWARD PROPAGATION

# Single-layer backward propagation
# Examples:
# params = [cross_entropy_derivative, probs_true]
# params = [relu_derivative, W_next, dL_dZ_next]
def backward_SL(X, Z, derivative_parameters):
    function_derivative = derivative_parameters[0]
    # Cross-entropy is always the outer layer: probs_pred = Z
    if function_derivative.__name__=='cross_entropy_derivative':
        probs_pred, probs_true = softmax(Z), derivative_parameters[1]
        dL_dZ = cross_entropy_derivative(probs_pred, probs_true)
    # ReLU is always an inner layer, so it needs:
    # W_next and dL_dZ_next from the "next" (in terms of forward propagation) step
    elif function_derivative.__name__=='relu_derivative':
        W_next, dL_dZ_next = derivative_parameters[1:]
        # Current step's Y is "next" step's X
        dL_dY = dL_dZ_next.dot(W_next.T)
        dL_dZ = dL_dY * relu_derivative(Z)
    else:
        print("Wrong derivative function!")
        exit()
    # This works for every layer: Z = W*X + b
    dL_dW = (X.T).dot(dL_dZ)
    dL_db = np.sum(dL_dZ, axis=0, keepdims=True)
    # Export derivatives
    return [dL_dZ, dL_dW, dL_db]

# Multi-layer backward propagation
def backward_ML(X_init, probs_true, parameters, forward_parameters):
    derivatives = []
    # Layers are counted in the direction of forward propagation
    # layer 0 is the initial data
    for layer in range(num_layers-1,-1,-1):
        # If it is the outer layer, apply cross_entroy_derivative
        # If it is an inner layer, apply relu_derivative with:
        # W_next = parameters[layer-1,1]
        # dL_dZ_next = derivatives[0,0]
        backward_parameters = \
            [cross_entropy_derivative, probs_true] if layer==num_layers-1 else \
            [relu_derivative, parameters[layer+1][0], derivatives[0][0]]
        X, Z = forward_parameters[layer][1], forward_parameters[layer+1][0]
        derivative = backward_SL(X, Z, backward_parameters)
        derivatives.insert(0,derivative)
    return derivatives


# Training function
def train(X, Y, parameters = None, \
          rate_init=0.05, num_epochs=300, lambda_reg=0.0001, momentum=0.9):

    # Assign default parameters
    if parameters is None:
        parameters = [[W.copy(), b.copy()] for (W,b) in parameters_init]

    # Probabilities of outputs
    probs_true = one_hot(Y)

    # Initialize velocities
    # velocities = [[v_W1,v_b1],[v_W2,v_b2],...]
    velocities = []
    for parameter in parameters:
        velocities.append([np.zeros_like(parameter[0]),np.zeros_like(parameter[1])])

    # Iterate over epochs
    for epoch in range(num_epochs):

        # Forward propagation
        forward_parameters = forward_ML(X, parameters)

        # Backward propagation
        derivatives = backward_ML(X, probs_true, parameters, forward_parameters)

        # Iterate over layers
        for layer in range(0,num_layers):
            # L2-regularization
            derivatives[layer][1] += lambda_reg*parameters[layer][0]
            # Re-calculate velocities
            velocities[layer][0] = momentum*velocities[layer][0] - rate_init*derivatives[layer][1]
            velocities[layer][1] = momentum*velocities[layer][1] - rate_init*derivatives[layer][2]
            # Perform gradient descent
            parameters[layer][0] += velocities[layer][0]
            parameters[layer][1] += velocities[layer][1]

        # Print out loss function values
        if epoch % 10 == 0:
            # Calculate loss
            loss = cross_entropy(forward_parameters[-1][1],probs_true)
            # Calculate accuracy
            accuracy = np.mean(np.argmax(forward_parameters[-1][1], axis=1)==Y)
            print(f"Epoch {epoch}, loss {loss:.4f}, accuracy {accuracy:.4f}")

    return parameters


In [ ]:
parameters_0 = train(X, Y)

In [ ]:
len(parameters_init)

In [ ]:
# Forward propagation: ReLU+softmax
# H = output of hidden layer (score)
def forward2(X, W1, b1, W2, b2):
    # First layer: ReLU
    Z1 = X.dot(W1) + b1
    H = relu(Z1)
    # Second layer: softmax
    Z2 = H.dot(W2) + b2
    probs_pred = softmax(Z2)
    return Z1, H, Z2, probs_pred

# Backward propagation
def backward2(X, Z1, H, W2, probs_pred, probs_true):
    # Gradient for softmax layer
    # d(loss)/d(z2)
    dL_dZ2 = cross_entropy_derivative(probs_pred, probs_true)
    # d(loss)/d(W2)
    dL_dW2 = (H.T).dot(dL_dZ2)
    dL_db2 = np.sum(dL_dZ2, axis=0, keepdims=True)
    # Gradient for ReLU layer
    dL_dH = dL_dZ2.dot(W2.T)
    dL_dZ1 = dL_dH * relu_derivative(Z1)
    dL_dW1 = (X.T).dot(dL_dZ1)
    dL_db1 = np.sum(dL_dZ1, axis=0, keepdims=True)
    return dL_dW2, dL_db2, dL_dW1, dL_db1

# Training function: ReLU+softmax
def train2(X, Y, rate_init=0.05, num_epochs=300, lambda_reg=0.0001, momentum=0.9, \
            W1 = np.sqrt(2.0/num_features)*np.random.randn(num_features,num_hidden), \
            b1 = np.full((1, num_hidden), 0.01), \
            W2 = np.sqrt(2.0/num_hidden)*np.random.randn(num_hidden,num_classes), \
            b2 = np.full((1, num_classes), 0.01)):

    # Initialize velocity terms
    v_W1, v_b1 = np.zeros_like(W1), np.zeros_like(b1)
    v_W2, v_b2 = np.zeros_like(W2), np.zeros_like(b2)

    # Probabilities of outputs
    probs_true = one_hot(Y)

    # Iterate over epochs
    for epoch in range(num_epochs):

        # Forward propagation
        Z1, H, Z2, probs_pred = forward2(X, W1, b1, W2, b2)

        # Backward propagation
        dL_dW2, dL_db2, dL_dW1, dL_db1 = backward2(X,Z1,H,W2,probs_pred,probs_true)

        # L2 regularization
        dL_dW1 += lambda_reg*W1
        dL_dW2 += lambda_reg*W2

        # Update momenta
        v_W1 = momentum*v_W1 - rate_init*dL_dW1
        v_b1 = momentum*v_b1 - rate_init*dL_db1
        v_W2 = momentum*v_W2 - rate_init*dL_dW2
        v_b2 = momentum*v_b2 - rate_init*dL_db2

        # Gradient descent
        W1 += v_W1
        b1 += v_b1
        W2 += v_W2
        b2 += v_b2

        # Print out loss function values
        if epoch % 10 == 0:
            # Calculate loss
            loss = cross_entropy(probs_pred,probs_true)
            # Calculate accuracy
            accuracy = np.mean(np.argmax(probs_pred, axis=1)==Y)
            print(f"Epoch {epoch}, loss {loss:.4f}, accuracy {accuracy:.4f}")

    return W1, b1, W2, b2

In [ ]:
W1_0, b1_0, W2_0, b2_0 = train2(X,Y)

In [ ]:
print("Inputs:\n",X.shape)
print("Outputs:\n",Y.shape,"\n")

scores = [[2,1,0],[2,3,5]]
print("Arbitrary valued scores:\n",scores)
print("Corresponding probabilities:\n",softmax(scores),"\n")

Y_true = [1,3]
Y_pred = [[1,2,4,5,0,0,0,1,2,1],[1,0,4,5,0,0,0,1,2,1]]
print("Model predictions (probabilities):\n",softmax(Y_pred))
print("Outputs (probabilities):\n",one_hot(Y_true))
print("Cross-entropy:\n",cross_entropy(one_hot(Y_true),softmax(Y_pred)))

In [ ]:
# Calculate predicted values.
# Outputs are classes with largest probabilities.
def predict(X, W1, b1, W2, b2):
    Z1 = X.dot(W1) + b1
    H = relu(Z1)
    Z2 = H.dot(W2) + b2
    probs_pred = softmax(Z2)
    return np.argmax(probs_pred, axis=1)

Y_pred = predict(X, W1_0, b1_0, W2_0, b2_0)

accuracy = np.mean(Y_pred == Y)
print("Training accuracy:", accuracy)

In [ ]:
# Calculate test values.
# Inputs:
# X = array(num_samples,num_features)
# (for each sample, pixel values in range [0,1], flattened 28x28-pixel image)
X_test = data_test[:,:]/255.0
# Mean subtraction: subtract average across dataset, to make features balanced
X_test = (X_test - np.mean(X_test, axis=0)) / (np.std(X_test, axis=0)+10**(-9))

# Outputs:
# Y = array(num_samples)
# (for each sample, correct class = digit)
Y_test = data_test[:,0].astype(int)


# Outputs are classes with largest probabilities.
def predict(X, W1, b1, W2, b2):
    Z1 = X.dot(W1) + b1
    H = relu(Z1)
    Z2 = H.dot(W2) + b2
    probs_pred = softmax(Z2)
    return np.argmax(probs_pred, axis=1)

Y_pred_test = predict(X_test, W1_0, b1_0, W2_0, b2_0)

#accuracy = np.mean(Y_pred == Y_test)
#print("Testing accuracy:", accuracy)

In [ ]:
# Checking test data

# Pick an image
image_num = 3
label = Y_pred_test[image_num]
image = data_test[image_num][:].reshape(28,28)

# Display the image and its label
print("Predicted number is: ",int(label))
plt.imshow(image, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
matrix = (W1_0.dot(W2_0))[:,8].reshape(28,28)
plt.imshow(matrix, cmap="gray")
plt.axis("off")
plt.show()

## Training test

In [ ]:
# This function performs model training.
# W and b are model parameters
def train(X, Y, rate=0.1, num_epochs=300, \
            W1 = 0.01*np.random.randn(num_features,num_hidden), \
            b1 = np.zeros((1,num_hidden)), \
            W2 = 0.01*np.random.randn(num_hidden,num_classes), \
            b2 = np.zeros((1,num_classes))):

    # Probabilities of outputs
    probs_true = one_hot(Y)

    # Iterate over epochs
    for epoch in range(num_epochs):

        # Forward calculation:

        # First layer: ReLU





        # Calculate arbitrary-valued scores (logits)
        logits = X.dot(W) + b
        # Convert logits into probabilities
        probs_pred = softmax(logits)

        # Calculate loss function
        loss = cross_entropy(probs_true, probs_pred)

        # Calculate gradients of loss function wrt model parameters
        # d(loss)/d(logits)
        gradients = (probs_pred - probs_true) / num_samples
        # d(loss)/dW
        dW = (X.T).dot(gradients)
        # d(loss)/db
        db = np.sum(gradients, axis=0, keepdims=True)

        # Update model parameters (gradient descent)
        W -= rate * dW
        b -= rate * db

        # Print out loss function values
        if epoch % 5 == 0: print(f"Epoch {epoch}, loss {loss:.4f}")

    # Return model parameters
    return W, b

# Train model
# Initial run:
W0, b0 = train(X,Y)
# Additional runs:
W0,b0 = train(X,Y,rate=0.01,num_epochs=1000,W=W0,b=b0)

### Test functions

In [ ]:
print("Inputs:\n",X.shape)
print("Outputs:\n",Y.shape,"\n")

scores = [[2,1,0],[2,3,5]]
print("Arbitrary valued scores:\n",scores)
print("Corresponding probabilities:\n",softmax(scores),"\n")

Y_true = [1,3]
Y_pred = [[1,2,4,5,0,0,0,1,2,1],[1,0,4,5,0,0,0,1,2,1]]
print("Model predictions (probabilities):\n",softmax(Y_pred))
print("Outputs (probabilities):\n",one_hot(Y_true))
print("Cross-entropy:\n",cross_entropy(one_hot(Y_true),softmax(Y_pred)))


### Training

In [ ]:
# This function performs model training.
# W and b are model parameters
def train(X, Y, rate=0.1, num_epochs=300, \
            W = np.random.randn(num_features,num_classes)*0.01, \
            b = np.zeros((1,num_classes))):

    # Probabilities of outputs
    probs_true = one_hot(Y)

    # Iterate over epochs
    for epoch in range(num_epochs):

        # Forward calculation:
        # Calculate arbitrary-valued scores (logits)
        logits = X.dot(W) + b
        # Convert logits into probabilities
        probs_pred = softmax(logits)

        # Calculate loss function
        loss = cross_entropy(probs_true, probs_pred)

        # Calculate gradients of loss function wrt model parameters
        # d(loss)/d(logits)
        gradients = (probs_pred - probs_true) / num_samples
        # d(loss)/dW
        dW = (X.T).dot(gradients)
        # d(loss)/db
        db = np.sum(gradients, axis=0, keepdims=True)

        # Update model parameters (gradient descent)
        W -= rate * dW
        b -= rate * db

        # Print out loss function values
        if epoch % 5 == 0: print(f"Epoch {epoch}, loss {loss:.4f}")

    # Return model parameters
    return W, b

# Train model
# Initial run:
W0, b0 = train(X,Y)
# Additional runs:
W0,b0 = train(X,Y,rate=0.01,num_epochs=1000,W=W0,b=b0)

In [ ]:
# Calculate predicted values.
# Outputs are classes with largest probabilities.
def predict(X, W, b):
    logits = X.dot(W) + b
    return np.argmax(logits, axis=1)

Y_pred = predict(X, W0, b0)

accuracy = np.mean(Y_pred == Y)
print("Training accuracy:", accuracy)

In [ ]:
# Explanation of train:

# Initialize matrices
# W = random matrix (784*10)
# b = bias, initially zeros (1*10)
num_classes = 10
N, D = X.shape
W = np.random.randn(D, num_classes) * 0.01
b = np.zeros((1, num_classes))

# Convert labels into probability vectors
answers = one_hot(Y.astype(int),num_classes)

# Calculate linear function from inputs
scores = X.dot(W)+b

# Convert scores to probabilities
probs = softmax(scores)

# Sum of probs = 1
#print("Sum of probs:",np.sum(probs[1]))

# Compare predicted probs with answers
loss = cross_entropy(answers,probs)

print("Loss:",loss)

# Main magic: gradient descent

# grad_scores = d(loss)/d(scores)
grad_scores = (probs-answers)/N

#print(grad_scores)

# Derivative d(loss)/dW
dW = (X.T).dot(grad_scores)

# Derivative d(loss)/db
db = np.sum(grad_scores, axis=0, keepdims=True)

In [ ]:
print(Y_pred[:20])
print(Y[:20].astype(int))

In [ ]:
W = W0